In [2]:
import tensorflow
import keras
import os
import pandas as pd
import h5py
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.models import Sequential, Model
from keras.layers.core import Dense, Activation
from keras.layers.recurrent import LSTM
from keras.layers import Dropout, Flatten, Convolution1D as Conv1D, Convolution2D as Conv2D
from keras.callbacks import EarlyStopping
from keras.layers import Bidirectional, Input
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.layers import MaxPooling1D, AveragePooling1D, BatchNormalization as BatchNorm
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold

### Load the dataset

In [3]:
path = os.getcwd()
print(path)
filename = '/encoding_A_freq100_chunks200.h5'

C:\Users\Anony\Desktop\final\keyboard\code


In [4]:
# h5f = h5py.File(path + filename,'r')
# X_train = h5f['train'][:]
# Y_train = h5f['target'][:]
# X_test = h5f['test'][:]
# Y_test = h5f['target_test'][:]
# h5f.close()

# print('Target shape', Y_train.shape)
# print('Train shape', X_train.shape)
# print('Target test shape', Y_test.shape)
# print('Test shape', X_test.shape)

h5f = h5py.File(path + filename,'r')
X_train = h5f['train'][:]
Y_train = h5f['target'][:]
h5f.close()

print('Target shape', Y_train.shape)
print('Train shape', X_train.shape)

Target shape (61696,)
Train shape (61696, 200, 1)


### Extend train set by flipping the data

In [5]:
extend_x = np.flip(X_train, axis=1)
extend_y = Y_train

X_train = np.concatenate((X_train, extend_x), axis=0)
Y_train = np.concatenate((Y_train, extend_y), axis=0)

print('Target shape', Y_train.shape)
print('Train shape', X_train.shape)
# print('Target test shape', Y_test.shape)
# print('Test shape', X_test.shape)

Target shape (123392,)
Train shape (123392, 200, 1)


### Build model - Inception with different kernel sizes

In [5]:
### Alternative model - 2 conv layers
np.random.seed(14)  # fix the random numbers generator state

batch_size = 64
pool_size = 30
strides = 30
# hidden_units = 15
input_shape = X_train.shape[1:]
nb_epochs = 30
nb_classes = 1
dropout = 0.05
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=3, verbose=1)
# sgd = SGD(lr=0.005, decay=1e-5, momentum=0.9, nesterov=True)

print('Build model...')

# model = Sequential()

# model.add(Conv1D(input_shape=input_shape, filters=64, kernel_size=50))
# model.add(BatchNorm())
# model.add(LeakyReLU(alpha=0.01))
# model.add(Dropout(dropout))

# model.add(Conv1D(filters=128, kernel_size=50))
# model.add(BatchNorm())
# model.add(LeakyReLU(alpha=0.01))
# model.add(MaxPooling1D(pool_size=pool_size, strides=strides))

# model.add(Flatten())
# model.add(Dropout(dropout))

# model.add(Dense(128))
# model.add(Activation('linear'))

# model.add(Dense(nb_classes))
# model.add(Activation('sigmoid'))

inp = Input(shape=input_shape)

tower_1 = Conv1D(filters=32, kernel_size=30, padding='same')(inp)
tower_1 = LeakyReLU(alpha=0.01)(tower_1)
tower_1 = Conv1D(filters=64, kernel_size=30, padding='same')(tower_1)
tower_1 = LeakyReLU(alpha=0.01)(tower_1)

tower_2 = Conv1D(filters=32, kernel_size=40, padding='same')(inp)
tower_2 = LeakyReLU(alpha=0.01)(tower_2)
tower_2 = Conv1D(filters=64, kernel_size=40, padding='same')(tower_2)
tower_2 = LeakyReLU(alpha=0.01)(tower_2)

tower_3 = Conv1D(filters=32, kernel_size=50, padding='same')(inp)
tower_3 = LeakyReLU(alpha=0.01)(tower_3)
tower_3 = Conv1D(filters=64, kernel_size=50, padding='same')(tower_3)
tower_3 = LeakyReLU(alpha=0.01)(tower_3)

middle = keras.layers.concatenate([tower_1, tower_2, tower_3], axis=2)
middle = BatchNorm()(middle)
middle = LeakyReLU(alpha=0.01)(middle)
middle = MaxPooling1D(pool_size=pool_size, strides=strides, padding='same')(middle)
middle = Flatten()(middle)

out = Dense(128, activation='linear')(middle)
out = Dense(1, activation='sigmoid')(out)

model = Model(inputs=inp, outputs=out)

model.compile(loss='binary_crossentropy', metrics=['binary_accuracy'], optimizer='adam')  # was adam (rmsprop alternative)

model.summary()

print("Train...")
history = model.fit(X_train, Y_train, batch_size=batch_size, epochs=nb_epochs, verbose=1, callbacks=[early_stopping], 
                   validation_split=0.15)

Build model...
Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 500, 1)       0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 500, 32)      992         input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_3 (Conv1D)               (None, 500, 32)      1312        input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_5 (Conv1D)               (None, 500, 32)      1632        input_1[0][0]                    
___________

### Test the model performance

In [6]:
Y_pred = model.predict(X_test)
print('Accuracy is', accuracy_score(Y_test, np.round(Y_pred)))
AUC = roc_auc_score(Y_test, Y_pred)
print('AUC is', AUC)
f1 = f1_score(Y_test, np.round(Y_pred))
print('F1-score is', f1)

Accuracy is 0.6327608982826949
AUC is 0.6974089475769089
F1-score is 0.5716486902927581


In [11]:
np.round(Y_pred)
# Y_pred

array([[1.],
       [1.],
       [1.],
       ...,
       [0.],
       [0.],
       [0.]], dtype=float32)

### Inception with dilated units

In [7]:
### Alternative model - 2 conv layers
np.random.seed(14)  # fix the random numbers generator state

batch_size = 64
pool_size = 30
strides = 30
# hidden_units = 15
input_shape = X_train.shape[1:]
nb_epochs = 30
nb_classes = 1
dropout = 0.05
# early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=3, verbose=1)
# sgd = SGD(lr=0.005, decay=1e-5, momentum=0.9, nesterov=True)

print('Build model...')

# model = Sequential()

# model.add(Conv1D(input_shape=input_shape, filters=64, kernel_size=50))
# model.add(BatchNorm())
# model.add(LeakyReLU(alpha=0.01))
# model.add(Dropout(dropout))

# model.add(Conv1D(filters=128, kernel_size=50))
# model.add(BatchNorm())
# model.add(LeakyReLU(alpha=0.01))
# model.add(MaxPooling1D(pool_size=pool_size, strides=strides))

# model.add(Flatten())
# model.add(Dropout(dropout))

# model.add(Dense(128))
# model.add(Activation('linear'))

# model.add(Dense(nb_classes))
# model.add(Activation('sigmoid'))

inp = Input(shape=input_shape)

tower_1 = Conv1D(filters=16, kernel_size=50, padding='same')(inp)
tower_1 = LeakyReLU(alpha=0.01)(tower_1)
tower_1 = Dropout(dropout)(tower_1)
tower_1 = Conv1D(filters=16, kernel_size=50, padding='same', dilation_rate=2)(tower_1)
tower_1 = LeakyReLU(alpha=0.01)(tower_1)
tower_1 = Dropout(dropout)(tower_1)
tower_1 = Conv1D(filters=16, kernel_size=50, padding='same', dilation_rate=3)(tower_1)
tower_1 = LeakyReLU(alpha=0.01)(tower_1)
tower_1 = Dropout(dropout)(tower_1)

tower_2 = Conv1D(filters=32, kernel_size=50, padding='same')(inp)
tower_2 = LeakyReLU(alpha=0.01)(tower_2)
tower_2 = Dropout(dropout)(tower_2)
tower_2 = Conv1D(filters=32, kernel_size=50, padding='same', dilation_rate=2)(tower_2)
tower_2 = LeakyReLU(alpha=0.01)(tower_2)
tower_2 = Dropout(dropout)(tower_2)
tower_2 = Conv1D(filters=32, kernel_size=50, padding='same', dilation_rate=3)(tower_2)
tower_2 = LeakyReLU(alpha=0.01)(tower_2)
tower_2 = Dropout(dropout)(tower_2)

tower_3 = Conv1D(filters=64, kernel_size=50, padding='same')(inp)
tower_3 = LeakyReLU(alpha=0.01)(tower_3)
tower_3 = Dropout(dropout)(tower_3)
tower_3 = Conv1D(filters=64, kernel_size=50, padding='same', dilation_rate=2)(tower_3)
tower_3 = LeakyReLU(alpha=0.01)(tower_3)
tower_3 = Conv1D(filters=64, kernel_size=50, padding='same', dilation_rate=3)(tower_3)
tower_3 = LeakyReLU(alpha=0.01)(tower_3)

middle = keras.layers.concatenate([tower_1, tower_2, tower_3], axis=2)
middle = BatchNorm()(middle)
middle = LeakyReLU(alpha=0.01)(middle)
middle = MaxPooling1D(pool_size=pool_size, strides=strides, padding='same')(middle)
middle = Flatten()(middle)
middle = Dropout(dropout)(middle)

out = Dense(128, activation='linear')(middle)
out = Dense(1, activation='sigmoid')(out)

model = Model(inputs=inp, outputs=out)

model.compile(loss='binary_crossentropy', metrics=['binary_accuracy'], optimizer='sgd')  # was adam (rmsprop alternative)

model.summary()

print("Train...")
history = model.fit(X_train, Y_train, batch_size=batch_size, epochs=nb_epochs, verbose=1,validation_split=0.15)

Build model...
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 200, 1)       0                                            
__________________________________________________________________________________________________
conv1d_10 (Conv1D)              (None, 200, 16)      816         input_2[0][0]                    
__________________________________________________________________________________________________
conv1d_13 (Conv1D)              (None, 200, 32)      1632        input_2[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu_11 (LeakyReLU)      (None, 200, 16)      0           conv1d_10[0][0]                  
______________________________________________________________________________________________

104883/104883 [==============================] - 30s 288us/step - loss: 0.6250 - binary_accuracy: 0.6310 - val_loss: 0.6805 - val_binary_accuracy: 0.5769- loss: 0.6262 - binary_accuracy: 0 - ETA: 9s - lo - ETA: 1s - loss: 0.6249 - binary_accuracy - ETA: 1s - loss: 0.62
Epoch 5/30
104883/104883 [==============================] - 30s 284us/step - loss: 0.6227 - binary_accuracy: 0.6316 - val_loss: 0.6394 - val_binary_accuracy: 0.6386
Epoch 6/30
104883/104883 [==============================] - 30s 285us/step - loss: 0.6205 - binary_accuracy: 0.6344 - val_loss: 0.6404 - val_binary_accuracy: 0.6271
Epoch 7/30
104883/104883 [==============================] - 30s 287us/step - loss: 0.6178 - binary_accuracy: 0.6373 - val_loss: 0.6502 - val_binary_accuracy: 0.6254
Epoch 8/30
104883/104883 [==============================] - 30s 286us/step - loss: 0.6161 - binary_accuracy: 0.6399 - val_loss: 0.6404 - val_binary_accuracy: 0.6366ss: 0.6169 - bi - ETA: 6s - loss:  - ETA: 4s 
Epoch 9/30
104883/104883 

In [8]:
Y_pred = model.predict(X_test)
print('Accuracy is', accuracy_score(Y_test, np.round(Y_pred)))
AUC = roc_auc_score(Y_test, Y_pred)
print('AUC is', AUC)
f1 = f1_score(Y_test, np.round(Y_pred))
print('F1-score is', f1)

NameError: name 'X_test' is not defined

### Cross-validation - Inception

In [6]:
# fix random seed for reproducibility
seed = 7
# define 10-fold cross validation test harness
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
acc_list = []
AUC_list = []
f1_list = []
loss = []

batch_size = 64
pool_size = 30
strides = 30
# hidden_units = 15
input_shape = X_train.shape[1:]
nb_epochs = 30
nb_classes = 1
dropout = 0.05
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=3, verbose=1)

X = X_train
Y = Y_train

for train, test in kfold.split(X, Y):
    
    X_train = X[train]
    Y_train = Y[train]
    X_test = X[test]
    Y_test = Y[test]
    
    # added flip of the training data
    extend_x = np.flip(X_train, axis=1)
    extend_y = Y_train

    X_train = np.concatenate((X_train, extend_x), axis=0)
    Y_train = np.concatenate((Y_train, extend_y), axis=0)
    
    print('Build model...')

    inp = Input(shape=input_shape)

    tower_1 = Conv1D(filters=16, kernel_size=50, padding='same')(inp)
    tower_1 = LeakyReLU(alpha=0.01)(tower_1)
    tower_1 = Dropout(dropout)(tower_1)
    tower_1 = Conv1D(filters=16, kernel_size=50, padding='same', dilation_rate=2)(tower_1)
    tower_1 = LeakyReLU(alpha=0.01)(tower_1)
    tower_1 = Dropout(dropout)(tower_1)
    tower_1 = Conv1D(filters=16, kernel_size=50, padding='same', dilation_rate=3)(tower_1)
    tower_1 = LeakyReLU(alpha=0.01)(tower_1)
    tower_1 = Dropout(dropout)(tower_1)

    tower_2 = Conv1D(filters=32, kernel_size=50, padding='same')(inp)
    tower_2 = LeakyReLU(alpha=0.01)(tower_2)
    tower_2 = Dropout(dropout)(tower_2)
    tower_2 = Conv1D(filters=32, kernel_size=50, padding='same', dilation_rate=2)(tower_2)
    tower_2 = LeakyReLU(alpha=0.01)(tower_2)
    tower_2 = Dropout(dropout)(tower_2)
    tower_2 = Conv1D(filters=32, kernel_size=50, padding='same', dilation_rate=3)(tower_2)
    tower_2 = LeakyReLU(alpha=0.01)(tower_2)
    tower_2 = Dropout(dropout)(tower_2)

    tower_3 = Conv1D(filters=64, kernel_size=50, padding='same')(inp)
    tower_3 = LeakyReLU(alpha=0.01)(tower_3)
    tower_3 = Dropout(dropout)(tower_3)
    tower_3 = Conv1D(filters=64, kernel_size=50, padding='same', dilation_rate=2)(tower_3)
    tower_3 = LeakyReLU(alpha=0.01)(tower_3)
    tower_3 = Conv1D(filters=64, kernel_size=50, padding='same', dilation_rate=3)(tower_3)
    tower_3 = LeakyReLU(alpha=0.01)(tower_3)

    middle = keras.layers.concatenate([tower_1, tower_2, tower_3], axis=2)
    middle = BatchNorm()(middle)
    middle = LeakyReLU(alpha=0.01)(middle)
    middle = MaxPooling1D(pool_size=pool_size, strides=strides, padding='same')(middle)
    middle = Flatten()(middle)
    middle = Dropout(dropout)(middle)

    out = Dense(128, activation='linear')(middle)
    out = Dense(1, activation='sigmoid')(out)

    model = Model(inputs=inp, outputs=out)

    model.compile(loss='binary_crossentropy', metrics=['binary_accuracy'], optimizer='adam')  # was adam (rmsprop alternative)

    model.summary()

    print("Train...")
    history = model.fit(X_train, Y_train, batch_size=batch_size, epochs=nb_epochs, verbose=1, callbacks=[early_stopping],
                        validation_split=0.15)

    Y_pred = model.predict(X_test)
    acc = accuracy_score(Y_test, np.round(Y_pred))
    print('Accuracy is', acc)
    AUC = roc_auc_score(Y_test, Y_pred)
    print('AUC is', AUC)
    f1 = f1_score(Y_test, np.round(Y_pred))
    print('F1-score is', f1)
    
    acc_list.append(acc)
    AUC_list.append(AUC)
    f1_list.append(f1)
    loss.append(np.mean(history.history['val_loss']))
    
print('Accuracy: ', np.mean(acc_list)*100)
print('AUC: ', np.mean(AUC_list))
print('f1-score: ', np.mean(f1_list))


Build model...
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 200, 1)       0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 200, 16)      816         input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_4 (Conv1D)               (None, 200, 32)      1632        input_1[0][0]                    
__________________________________________________________________________________________________
leaky_

Train on 188788 samples, validate on 33316 samples
Epoch 1/30
  4864/188788 [..............................] - ETA: 17:58 - loss: 1.0372 - binary_accuracy: 0.5362

KeyboardInterrupt: 

In [ ]:
AUC_list

### Cross-validation - dilated Inception

In [34]:
# fix random seed for reproducibility
seed = 7
# define 10-fold cross validation test harness
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
acc_list = []
AUC_list = []
f1_list = []
loss = []

batch_size = 64
pool_size = 30
strides = 30
# hidden_units = 15
input_shape = X_train.shape[1:]
nb_epochs = 30
nb_classes = 1
dropout = 0.05
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=3, verbose=1)

X = X_train
Y = Y_train

for train, test in kfold.split(X, Y):
    
    X_train = X[train]
    Y_train = Y[train]
    X_test = X[test]
    Y_test = Y[test]
    
    # added flip of the training data
#     extend_x = np.flip(X_train, axis=1)
#     extend_y = Y_train

#     X_train = np.concatenate((X_train, extend_x), axis=0)
#     Y_train = np.concatenate((Y_train, extend_y), axis=0)
    
    print('Build model...')

    inp = Input(shape=input_shape)

    tower_1 = Conv1D(filters=16, kernel_size=50, padding='same')(inp)
    tower_1 = LeakyReLU(alpha=0.01)(tower_1)
    tower_1 = Dropout(dropout)(tower_1)
    tower_1 = Conv1D(filters=16, kernel_size=50, padding='same', dilation_rate=2)(tower_1)
    tower_1 = LeakyReLU(alpha=0.01)(tower_1)
    tower_1 = Dropout(dropout)(tower_1)
    tower_1 = Conv1D(filters=16, kernel_size=50, padding='same', dilation_rate=3)(tower_1)
    tower_1 = LeakyReLU(alpha=0.01)(tower_1)
    tower_1 = Dropout(dropout)(tower_1)

    tower_2 = Conv1D(filters=32, kernel_size=50, padding='same')(inp)
    tower_2 = LeakyReLU(alpha=0.01)(tower_2)
    tower_2 = Dropout(dropout)(tower_2)
    tower_2 = Conv1D(filters=32, kernel_size=50, padding='same', dilation_rate=2)(tower_2)
    tower_2 = LeakyReLU(alpha=0.01)(tower_2)
    tower_2 = Dropout(dropout)(tower_2)
    tower_2 = Conv1D(filters=32, kernel_size=50, padding='same', dilation_rate=3)(tower_2)
    tower_2 = LeakyReLU(alpha=0.01)(tower_2)
    tower_2 = Dropout(dropout)(tower_2)

    tower_3 = Conv1D(filters=64, kernel_size=50, padding='same')(inp)
    tower_3 = LeakyReLU(alpha=0.01)(tower_3)
    tower_3 = Dropout(dropout)(tower_3)
    tower_3 = Conv1D(filters=64, kernel_size=50, padding='same', dilation_rate=2)(tower_3)
    tower_3 = LeakyReLU(alpha=0.01)(tower_3)
    tower_3 = Conv1D(filters=64, kernel_size=50, padding='same', dilation_rate=3)(tower_3)
    tower_3 = LeakyReLU(alpha=0.01)(tower_3)

    middle = keras.layers.concatenate([tower_1, tower_2, tower_3], axis=2)
    middle = BatchNorm()(middle)
    middle = LeakyReLU(alpha=0.01)(middle)
    middle = MaxPooling1D(pool_size=pool_size, strides=strides, padding='same')(middle)
    middle = Flatten()(middle)
    middle = Dropout(dropout)(middle)

    out = Dense(128, activation='linear')(middle)
    out = Dense(1, activation='sigmoid')(out)

    model = Model(inputs=inp, outputs=out)

    model.compile(loss='binary_crossentropy', metrics=['binary_accuracy'], optimizer='adam')  # was adam (rmsprop alternative)

    model.summary()

    print("Train...")
    history = model.fit(X_train, Y_train, batch_size=batch_size, epochs=nb_epochs, verbose=1, callbacks=[early_stopping],
                        validation_split=0.15)

    Y_pred = model.predict(X_test)
    acc = accuracy_score(Y_test, np.round(Y_pred))
    print('Accuracy is', acc)
    AUC = roc_auc_score(Y_test, Y_pred)
    print('AUC is', AUC)
    f1 = f1_score(Y_test, np.round(Y_pred))
    print('F1-score is', f1)
    
    acc_list.append(acc)
    AUC_list.append(AUC)
    f1_list.append(f1)
    loss.append(np.mean(history.history['val_loss']))
    
print('Accuracy: ', np.mean(acc_list)*100)
print('AUC: ', np.mean(AUC_list))
print('f1-score: ', np.mean(f1_list))


Build model...
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_62 (InputLayer)           (None, 500, 1)       0                                            
__________________________________________________________________________________________________
conv1d_550 (Conv1D)             (None, 500, 16)      816         input_62[0][0]                   
__________________________________________________________________________________________________
conv1d_553 (Conv1D)             (None, 500, 32)      1632        input_62[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_611 (LeakyReLU)     (None, 500, 16)      0           conv1d_550[0][0]                 
______________________________________________________________________________________________

11325/11325 [==============================] - 21s 2ms/step - loss: 1.8035 - binary_accuracy: 0.5571 - val_loss: 1.1180 - val_binary_accuracy: 0.3802
Epoch 2/30
11325/11325 [==============================] - 7s 579us/step - loss: 0.6883 - binary_accuracy: 0.6291 - val_loss: 0.7757 - val_binary_accuracy: 0.4622
Epoch 3/30
11325/11325 [==============================] - 7s 578us/step - loss: 0.6337 - binary_accuracy: 0.6502 - val_loss: 0.7156 - val_binary_accuracy: 0.6078
Epoch 4/30
11325/11325 [==============================] - 7s 578us/step - loss: 0.6073 - binary_accuracy: 0.6650 - val_loss: 0.7563 - val_binary_accuracy: 0.4887
Epoch 5/30
11325/11325 [==============================] - 7s 579us/step - loss: 0.6100 - binary_accuracy: 0.6695 - val_loss: 0.7454 - val_binary_accuracy: 0.5033
Epoch 6/30
11325/11325 [==============================] - 7s 577us/step - loss: 0.5841 - binary_accuracy: 0.6834 - val_loss: 0.7793 - val_binary_accuracy: 0.5893
Epoch 00006: early stopping
Accuracy is 

Train on 11326 samples, validate on 1999 samples
Epoch 1/30
11326/11326 [==============================] - 22s 2ms/step - loss: 1.4871 - binary_accuracy: 0.5885 - val_loss: 1.0863 - val_binary_accuracy: 0.6373
Epoch 2/30
11326/11326 [==============================] - 7s 579us/step - loss: 0.6808 - binary_accuracy: 0.6336 - val_loss: 0.7751 - val_binary_accuracy: 0.5188
Epoch 3/30
11326/11326 [==============================] - 7s 578us/step - loss: 0.6571 - binary_accuracy: 0.6465 - val_loss: 0.7978 - val_binary_accuracy: 0.5618
Epoch 4/30
11326/11326 [==============================] - 7s 579us/step - loss: 0.6216 - binary_accuracy: 0.6617 - val_loss: 0.7329 - val_binary_accuracy: 0.4682
Epoch 5/30
11326/11326 [==============================] - 7s 581us/step - loss: 0.5887 - binary_accuracy: 0.6766 - val_loss: 0.8049 - val_binary_accuracy: 0.5568
Epoch 6/30
11326/11326 [==============================] - 7s 578us/step - loss: 0.5792 - binary_accuracy: 0.6847 - val_loss: 0.9173 - val_bina

Train on 11326 samples, validate on 1999 samples
Epoch 1/30
11326/11326 [==============================] - 22s 2ms/step - loss: 2.1902 - binary_accuracy: 0.5462 - val_loss: 0.7107 - val_binary_accuracy: 0.6193
Epoch 2/30
11326/11326 [==============================] - 7s 578us/step - loss: 0.6934 - binary_accuracy: 0.6214 - val_loss: 0.9256 - val_binary_accuracy: 0.3442
Epoch 3/30
11326/11326 [==============================] - 7s 578us/step - loss: 0.6474 - binary_accuracy: 0.6413 - val_loss: 0.6609 - val_binary_accuracy: 0.5833
Epoch 4/30
11326/11326 [==============================] - 7s 581us/step - loss: 0.6378 - binary_accuracy: 0.6535 - val_loss: 0.7566 - val_binary_accuracy: 0.5858
Epoch 5/30
11326/11326 [==============================] - 6s 567us/step - loss: 0.6082 - binary_accuracy: 0.6703 - val_loss: 0.7072 - val_binary_accuracy: 0.4887
Epoch 6/30
11326/11326 [==============================] - 6s 560us/step - loss: 0.5938 - binary_accuracy: 0.6743 - val_loss: 0.8520 - val_bina

Train on 11327 samples, validate on 1999 samples
Epoch 1/30
11327/11327 [==============================] - 22s 2ms/step - loss: 2.3131 - binary_accuracy: 0.5465 - val_loss: 1.0552 - val_binary_accuracy: 0.6048
Epoch 2/30
11327/11327 [==============================] - 6s 560us/step - loss: 0.7168 - binary_accuracy: 0.6234 - val_loss: 0.7172 - val_binary_accuracy: 0.6243
Epoch 3/30
11327/11327 [==============================] - 6s 557us/step - loss: 0.6280 - binary_accuracy: 0.6535 - val_loss: 0.7157 - val_binary_accuracy: 0.5088
Epoch 4/30
11327/11327 [==============================] - 6s 558us/step - loss: 0.6043 - binary_accuracy: 0.6642 - val_loss: 0.7304 - val_binary_accuracy: 0.5678
Epoch 5/30
11327/11327 [==============================] - 6s 559us/step - loss: 0.5874 - binary_accuracy: 0.6785 - val_loss: 0.7607 - val_binary_accuracy: 0.5663
Epoch 00005: early stopping
Accuracy is 0.6547297297297298
AUC is 0.7121329437545654
F1-score is 0.6362989323843415
Build model...
___________

Train on 11327 samples, validate on 1999 samples
Epoch 1/30
11327/11327 [==============================] - 22s 2ms/step - loss: 1.4421 - binary_accuracy: 0.5754 - val_loss: 0.9688 - val_binary_accuracy: 0.6333
Epoch 2/30
11327/11327 [==============================] - 6s 559us/step - loss: 0.7007 - binary_accuracy: 0.6229 - val_loss: 0.7608 - val_binary_accuracy: 0.6453
Epoch 3/30
11327/11327 [==============================] - 6s 561us/step - loss: 0.6829 - binary_accuracy: 0.6360 - val_loss: 0.7206 - val_binary_accuracy: 0.5728
Epoch 4/30
11327/11327 [==============================] - 6s 562us/step - loss: 0.6238 - binary_accuracy: 0.6554 - val_loss: 0.8904 - val_binary_accuracy: 0.5733
Epoch 5/30
11327/11327 [==============================] - 6s 560us/step - loss: 0.6035 - binary_accuracy: 0.6657 - val_loss: 0.6825 - val_binary_accuracy: 0.5868
Epoch 6/30
11327/11327 [==============================] - 6s 562us/step - loss: 0.5896 - binary_accuracy: 0.6769 - val_loss: 0.7979 - val_bina

Train on 11327 samples, validate on 1999 samples
Epoch 1/30
11327/11327 [==============================] - 22s 2ms/step - loss: 1.0808 - binary_accuracy: 0.5766 - val_loss: 0.9084 - val_binary_accuracy: 0.3997
Epoch 2/30
11327/11327 [==============================] - 6s 560us/step - loss: 0.6821 - binary_accuracy: 0.6225 - val_loss: 0.7271 - val_binary_accuracy: 0.6233
Epoch 3/30
11327/11327 [==============================] - 6s 562us/step - loss: 0.6388 - binary_accuracy: 0.6428 - val_loss: 0.6744 - val_binary_accuracy: 0.6098
Epoch 4/30
11327/11327 [==============================] - 6s 564us/step - loss: 0.6120 - binary_accuracy: 0.6643 - val_loss: 0.7581 - val_binary_accuracy: 0.5673
Epoch 5/30
11327/11327 [==============================] - 6s 561us/step - loss: 0.5960 - binary_accuracy: 0.6759 - val_loss: 0.8150 - val_binary_accuracy: 0.5638
Epoch 6/30
11327/11327 [==============================] - 6s 564us/step - loss: 0.5880 - binary_accuracy: 0.6819 - val_loss: 0.9612 - val_bina

Train on 11327 samples, validate on 1999 samples
Epoch 1/30
11327/11327 [==============================] - 23s 2ms/step - loss: 1.8006 - binary_accuracy: 0.5813 - val_loss: 0.7155 - val_binary_accuracy: 0.6608
Epoch 2/30
11327/11327 [==============================] - 7s 586us/step - loss: 0.7423 - binary_accuracy: 0.6296 - val_loss: 1.3527 - val_binary_accuracy: 0.5463
Epoch 3/30
11327/11327 [==============================] - 7s 591us/step - loss: 0.6335 - binary_accuracy: 0.6514 - val_loss: 0.6688 - val_binary_accuracy: 0.6183
Epoch 4/30
11327/11327 [==============================] - 7s 589us/step - loss: 0.6194 - binary_accuracy: 0.6644 - val_loss: 0.6719 - val_binary_accuracy: 0.6278
Epoch 5/30
11327/11327 [==============================] - 7s 588us/step - loss: 0.5916 - binary_accuracy: 0.6745 - val_loss: 0.8629 - val_binary_accuracy: 0.4762
Epoch 6/30
11327/11327 [==============================] - 7s 590us/step - loss: 0.5852 - binary_accuracy: 0.6801 - val_loss: 0.6998 - val_bina

Train on 11327 samples, validate on 1999 samples
Epoch 1/30
11327/11327 [==============================] - 23s 2ms/step - loss: 1.9108 - binary_accuracy: 0.5633 - val_loss: 1.0247 - val_binary_accuracy: 0.6553
Epoch 2/30
11327/11327 [==============================] - 7s 581us/step - loss: 0.6791 - binary_accuracy: 0.6207 - val_loss: 0.7271 - val_binary_accuracy: 0.6343
Epoch 3/30
11327/11327 [==============================] - 7s 590us/step - loss: 0.6462 - binary_accuracy: 0.6346 - val_loss: 0.6948 - val_binary_accuracy: 0.6663
Epoch 4/30
11327/11327 [==============================] - 7s 585us/step - loss: 0.6289 - binary_accuracy: 0.6469 - val_loss: 0.7622 - val_binary_accuracy: 0.5778
Epoch 5/30
11327/11327 [==============================] - 7s 587us/step - loss: 0.6118 - binary_accuracy: 0.6624 - val_loss: 0.7511 - val_binary_accuracy: 0.5558
Epoch 6/30
11327/11327 [==============================] - 7s 588us/step - loss: 0.5991 - binary_accuracy: 0.6733 - val_loss: 0.7463 - val_bina

Train on 11327 samples, validate on 1999 samples
Epoch 1/30
11327/11327 [==============================] - 24s 2ms/step - loss: 1.9938 - binary_accuracy: 0.5679 - val_loss: 0.8683 - val_binary_accuracy: 0.6408
Epoch 2/30
11327/11327 [==============================] - 7s 586us/step - loss: 0.6795 - binary_accuracy: 0.6312 - val_loss: 0.8442 - val_binary_accuracy: 0.6053
Epoch 3/30
11327/11327 [==============================] - 7s 589us/step - loss: 0.6414 - binary_accuracy: 0.6485 - val_loss: 0.7125 - val_binary_accuracy: 0.5633
Epoch 4/30
11327/11327 [==============================] - 7s 587us/step - loss: 0.6142 - binary_accuracy: 0.6665 - val_loss: 0.7942 - val_binary_accuracy: 0.5123
Epoch 5/30
11327/11327 [==============================] - 7s 588us/step - loss: 0.6105 - binary_accuracy: 0.6673 - val_loss: 0.7004 - val_binary_accuracy: 0.4857
Epoch 6/30
11327/11327 [==============================] - 7s 588us/step - loss: 0.5969 - binary_accuracy: 0.6740 - val_loss: 0.7205 - val_bina

In [35]:
AUC_list

[0.6689541251655038,
 0.5953511412705957,
 0.723737097421308,
 0.5728106649159281,
 0.7121329437545654,
 0.7156482834185538,
 0.7080204528853178,
 0.7462353907962016,
 0.6991042731921111,
 0.7129519722425128]